---
layout: post
title: "Entropy와 Gini계수"
author: "Chanjun Kim"
categories: Data분석
tags: [DecisionTree, 의사결정나무, 불순도, Entropy와, Gini, 엔트로피, 지니계수, InformationGain, information]
image: 03_entropy_gini.png
---

## **목적**
- 지난번 포스팅에 ensemble 모델에 관하여 이야기하면서 약한 모형으로 의사결정나무를 많이 사용하는 것을 알 수 있었습니다. 이번에는 의사결정 나무를 만들기 위하여 사용되는 Entropy와 gini index에 대해서 알아보도록 하겠습니다.
<br/>
<br/>


### **트리 구축의 원칙**
![Oops](https://mblogthumb-phinf.pstatic.net/MjAxODEyMDlfMjYz/MDAxNTQ0MzY1Njc0OTY4.hFiUNsT9cpXJORVg3QGSLdx1F78bgZbOktHa3e7emnwg.P0iA6eeREGDfir58hS-s8ZMOak_P5-qHPPaR_bJePhwg.PNG.ehdrndd/image.png?type=w800)
> 출처 : https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=ehdrndd&logNo=221158124011
- 결정 트리를 구축할 때는 Occamm의 면도날처럼 데이터의 특성을 가장 잘 반영하는 간단한 가설을 먼저 채택하도록 되어있습니다. 어떻게 간단하고 합리적인 트리를 만들 수 있을 지 알아보겠습니다.
<br>
<br>

---

### **1. 결정 트리**
의사결정나무를 효율적으로 만들기 위해서는 변수의 기준에 따라 불순도/불확실성을 낮추는 방식으로 선택하여 만들게 됩니다.<br>
이에 불순도(Impurity) / 불확실성(Uncertainty)를 감소하는 것을 Information gain이라고 하며 이것을 최소화시키기 위하여 Gini Index와 Entropy라는 개념을 사용하게 되고 의사결정 나무의 종류에 따라 다르게 쓰입니다.<br>
sklearn에서 default로 쓰이는 건 gini계수이며 이는 CART(Classificatioin And Regression Tree)에 쓰입니다.<br>
ID3 그리고 이것을 개선한 C4.5, C5.0에서는 Entropy를 계산한다고 합니다. <br>
CART tree는 항상 2진 분류를 하는 방식으로 나타나며, Entropy 혹은 Entropy 기반으로 계산되는 Information gain으로 계산되며 다중 분리가 됩니다. <br>

- Gini계수와 Entropy 모두 높을수록 불순도가 높아져 분류를 하기 어렵습니다. <br>
![Oops](http://i.imgur.com/n3MVwHW.png)

|비 고|ID3|C4.5, C5|CART|
|:---:|:---:|:---:|:---:|
|평가지수|Entropy|Information gain|Gini Index(범주), 분산의 차이(수치)|
|분리방식|다지분리|다지분리(범주) 및 이진분리(수치)|항상2진 분리|
|비고|수치형 데이터 못 다룸|||

<br>
<br>
> 출처/참고자료 : https://ko.wikipedia.org/wiki/%EA%B2%B0%EC%A0%95_%ED%8A%B8%EB%A6%AC_%ED%95%99%EC%8A%B5%EB%B2%95 <br>
> 출처/참고자료 : https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=trashx&logNo=60099037740 <br>
> 출처/참고자료 : https://ratsgo.github.io/machine%20learning/2017/03/26/tree/

---

### **1. Gini Index**
일단 sklearn의 DecisionTreeClassifier의 default 값인 Gini 계수에 대해서 먼저 설명하겠습니다. <br> 

우선 Gini index의 공식입니다. <br>

- 영역의 데이터 비율을 제곱하여 더한 값을 1에서 빼주게 된다.
![Oops](https://blog.kakaocdn.net/dn/uwQUP/btquI1QZpzV/zg919kn8JXk2n0bWovsYkk/img.png) <br>
<br>
- 두개 영역 이상이 되면 비율의 제곱의 비율을 곱하여 1에서 빼주게 된다.
![Oops](https://blog.kakaocdn.net/dn/cfpPHK/btquKIXbRpq/5WF8UDRqrRAG5itVMx1oW0/img.png)
> 출처 : https://soobarkbar.tistory.com/17

In [154]:
import os
import sys

import math
import numpy as np
import pandas as pd
import scipy

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

import matplotlib as mpl
from matplotlib import pyplot as plt

%matplotlib inline

In [11]:
tennis = pd.read_csv("data/tennis.csv", index_col = "Day")
tennis

,Outlook,Temperature,Humidity,Wind,PlayTennis
Day,,,,,
D1,Sunny,Hot,High,Weak,No
D2,Sunny,Hot,High,Strong,No
D3,Overcast,Hot,High,Weak,Yes
D4,Rain,Mild,High,Weak,Yes
D5,Rain,Cool,Normal,Weak,Yes
D6,Rain,Cool,Normal,Strong,No
D7,Overcast,Cool,Normal,Strong,Yes
D8,Sunny,Mild,High,Weak,No
D9,Sunny,Cool,Normal,Weak,Yes


- 위와 같은 데이터가 있다고 할 때, 우리는 어떤 요인이 가장 확실한(불확실성이 적은) 변수일지 생각을 하고 트리를 만들어야합니다.

In [122]:
def gini_nontsplit(df, y_col, col) :
    Ys = df[y_col].value_counts()
    total_row = len(df)
    return 1 - np.sum([np.square(len(df[df[y_col] == y]) / total_row) for y in Ys.index])

In [124]:
gini_nontsplit(tennis, "PlayTennis", "Outlook")

0.4591836734693877

In [125]:
def gini_split(df, y_col, col, feature) :
    r1 = len(df[df[col] == feature])
    Y1 = dict(df[df[col] != feature][y_col].value_counts())
    r2 = len(df[df[col] != feature])
    Y2 = dict(df[df[col] != feature][y_col].value_counts())
    
    ratio = r1 / (r1 + r2)
    gi1 = 1 - np.sum([np.square(len(df[(df[col] == feature) & (df[y_col] == x)]) / r1) for x, y in Y1.items()])
    gi2 = 1 - np.sum([np.square(len(df[(df[col] != feature) & (df[y_col] == x)]) / r2) for x, y in Y2.items()])
    
    return (ratio * gi1) + ((1-ratio) * gi2)

In [126]:
gini_split(tennis, "PlayTennis", "Outlook", "Sunny")

0.3936507936507936

In [146]:
to_gini = {x : list(tennis[x].unique()) for x in ["Outlook", "Temperature", "Humidity", "Wind"]}
to_gini

{'Outlook': ['Sunny', 'Overcast', 'Rain'],
 'Temperature': ['Hot', 'Mild', 'Cool'],
 'Humidity': ['High', 'Normal'],
 'Wind': ['Weak', 'Strong']}

In [127]:
y_col = "PlayTennis"

In [143]:
[f"col : {idx}, split_feature : {v} : gini_index = {gini_split(tennis, y_col, idx, v)}" for idx, val in to_gini.items() for v in val]

['col : Outlook, split_feature : Sunny : gini_index = 0.3936507936507936',
 'col : Outlook, split_feature : Overcast : gini_index = 0.35714285714285715',
 'col : Outlook, split_feature : Rain : gini_index = 0.4571428571428571',
 'col : Temperature, split_feature : Hot : gini_index = 0.4428571428571429',
 'col : Temperature, split_feature : Mild : gini_index = 0.4583333333333333',
 'col : Temperature, split_feature : Cool : gini_index = 0.45',
 'col : Humidity, split_feature : High : gini_index = 0.3673469387755103',
 'col : Humidity, split_feature : Normal : gini_index = 0.3673469387755103',
 'col : Wind, split_feature : Weak : gini_index = 0.4285714285714286',
 'col : Wind, split_feature : Strong : gini_index = 0.42857142857142855']

In [147]:
gini_df = pd.DataFrame([[idx, v, gini_split(tennis, y_col, idx, v)] for idx, val in to_gini.items() for v in val], columns = ["cat1", "cat2", "gini"])

In [152]:
gini_df.iloc[gini_df["gini"].argmin()]

cat1     Outlook
cat2    Overcast
gini    0.357143
Name: 1, dtype: object

In [179]:
import graphviz
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [184]:
feature = ["Outlook", "Temperature", "Humidity", "Wind"]

In [185]:
["Outlook" + x for x in tennis["Outlook"].unique()]

['OutlookSunny', 'OutlookOvercast', 'OutlookRain']

In [188]:
oe = OneHotEncoder()

In [193]:
oe.fit_transform(tennis[["Outlook"]]).toarray()

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [195]:
tennis

,Outlook,Temperature,Humidity,Wind,PlayTennis
Day,,,,,
D1,Sunny,Hot,High,Weak,No
D2,Sunny,Hot,High,Strong,No
D3,Overcast,Hot,High,Weak,Yes
D4,Rain,Mild,High,Weak,Yes
D5,Rain,Cool,Normal,Weak,Yes
D6,Rain,Cool,Normal,Strong,No
D7,Overcast,Cool,Normal,Strong,Yes
D8,Sunny,Mild,High,Weak,No
D9,Sunny,Cool,Normal,Weak,Yes


In [ ]:
["Outlook", "Temperature", "Humidity", "Wind"]

In [197]:
tmp1 = pd.DataFrame(oe.fit_transform(tennis[["Outlook"]]).toarray(), columns = ["Outlook_" + x for x in ["Overcast", "Rain", "Sunny"]])

In [ ]:
tmp2 = pd.DataFrame(oe.fit_transform(tennis[["Outlook"]]).toarray(), columns = ["Outlook_" + x for x in ["Overcast", "Rain", "Sunny"]])

In [182]:
la_encode.fit_transform(tennis[["Outlook", "Temperature", "Humidity", "Wind"]])

ValueError: bad input shape (14, 4)

In [178]:
oh_encode.fit_transform(tennis[["Outlook"]]).toarray()

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [176]:
dt = DecisionTreeClassifier()
dt.fit(tennis[["Outlook", "Temperature", "Humidity", "Wind"]], tennis[y_col])

ValueError: could not convert string to float: 'Sunny'

In [175]:
print(tree.export_text(dt))

|--- feature_0 <= 0.50
|   |--- feature_6 <= 0.50
|   |   |--- feature_9 <= 0.50
|   |   |   |--- feature_3 <= 0.50
|   |   |   |   |--- class: Yes
|   |   |   |--- feature_3 >  0.50
|   |   |   |   |--- class: No
|   |   |--- feature_9 >  0.50
|   |   |   |--- class: Yes
|   |--- feature_6 >  0.50
|   |   |--- feature_2 <= 0.50
|   |   |   |--- feature_9 <= 0.50
|   |   |   |   |--- class: No
|   |   |   |--- feature_9 >  0.50
|   |   |   |   |--- class: Yes
|   |   |--- feature_2 >  0.50
|   |   |   |--- class: No
|--- feature_0 >  0.50
|   |--- class: Yes



![Oops](https://wikimedia.org/api/rest_v1/media/math/render/svg/57a7ee6d08213c28f3f633229d26d1f82ea71ba6)

<br>
<br>

---
<br>

참고 자료 : [https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=ehdrndd&logNo=221158124011](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=ehdrndd&logNo=221158124011)